<h1>CycleGAN for Style Transfer</h1>
<h3> Imports </h3>

In [1]:
#Python imports
import torch
import torch.nn as tnn
from torch.autograd import Variable
import sys
import itertools
from torch.utils.data import DataLoader
from Dataset import ImageSet
import datetime
import torchvision.utils as tv
import os
import matplotlib.pyplot as plt
import numpy as np

#Import network elements
import network_elements as net

#stdout code blocks reload's print statement
#prevstd = sys.stdout
#sys.stdout = None
reload(net)
print("Reloading net")
#sys.stdout = prevstd

Reloading net


<h3>Set parameters</h3>

In [2]:
#The number of channels in the input image
im_in_channels = 3

#The number of channels in the output image
im_out_channels = 3

#The size of the largest size of the input image
im_size = 256

LEARNING_RATE = 0.0002

LR_DECAY_START = 100
LR_DECAY_END = 200
NUM_EPOCHS = 200

load_partial_net = True
CURR_EPOCH = 0

<h3>Import dataset</h3>

In [3]:
imageSet = ImageSet();
dataset = "horse2zebra"
imageSet.loadData(dataset, 'train', im_size)
imgLoader = DataLoader(imageSet,shuffle=True)

./datasets/horse2zebra
Finished loading data


<h3>Define the CycleGAN network</h3>

In [4]:
#G : X->Y
G = net.Mapping(im_in_channels, im_out_channels, im_size)

#F : Y->X
F = net.Mapping(im_out_channels, im_out_channels, im_size)

#G mapping discriminator
Dx = net.Discriminator(im_in_channels)

#F mapping discriminator
Dy = net.Discriminator(im_out_channels)

#Initialize CUDA implementations
G.cuda();
F.cuda();
Dx.cuda();
Dy.cuda();

#Define losses
cycle_loss = tnn.L1Loss(); #Cycle-consistency loss
gan_loss = tnn.MSELoss(); #Adversarial loss
identity_loss = tnn.L1Loss(); #Prevents image tinting

<h3>Initialize weights for each network</h3>

In [5]:
# norm_weights function applies Gaussian norm weights with mean 0 and stddev 0.02
G.apply(net.conv_norm_weights);
F.apply(net.conv_norm_weights);
Dx.apply(net.conv_norm_weights);
Dy.apply(net.conv_norm_weights);

<h3>Set optimizer parameters</h3>

In [6]:
#simplified version has no pooling, paper does not, paper code does...
# REMOVE

gan_opt = torch.optim.Adam(itertools.chain(G.parameters(), F.parameters()),
                           lr = LEARNING_RATE)
dx_opt = torch.optim.Adam(Dx.parameters(), lr = LEARNING_RATE)
dy_opt = torch.optim.Adam(Dy.parameters(), lr = LEARNING_RATE)

The paper uses Adam optimizer with a learning rate of 0.0002 for the first 100 epochs that linearly decays to 0 over the next 100 epochs.

<h3> Initialize training variables </h3>

In [7]:
x = torch.cuda.FloatTensor(1, im_in_channels, im_size, im_size)
y = torch.cuda.FloatTensor(1, im_out_channels, im_size, im_size)

# Variable wrapper is depricated in >0.4
dReal = Variable(torch.cuda.FloatTensor(1).fill_(1.0), requires_grad = False)
dFake = Variable(torch.cuda.FloatTensor(1).fill_(0.0), requires_grad = False)

buffX_Y = net.MapBuffer()
buffY_X = net.MapBuffer()

if not os.path.isdir("./img/" + dataset + "/"):
    os.mkdir("./img/" + dataset + '/')
    
if not os.path.isdir("./log/" + dataset + "/"):
    os.mkdir("./log/" + dataset + "/")
    
if os.path.isfile("./log/" + dataset + "/loss.log") and not load_partial_net:    
    os.remove("./log/" + dataset + "/loss.log")
    
logfile = open("./log/" + dataset + "/loss.log","a")

<h3> Load Partial Net </h3>

In [8]:
if (load_partial_net):
    if os.path.isfile("./model/" + dataset + "/model_info.txt"):
        model_file = open("./model/" + dataset + "/model_info.txt", "r")
        CURR_EPOCH = int(model_file.read())
        model_file.close()
    G.load_state_dict(torch.load('./model/' + dataset + '/G.data'))
    F.load_state_dict(torch.load('./model/' + dataset + '/F.data'))
    Dx.load_state_dict(torch.load('./model/' + dataset + '/Dx.data'))
    Dy.load_state_dict(torch.load('./model/' + dataset + '/Dy.data'))
    gan_opt.load_state_dict(torch.load('./model/' + dataset + '/gan_opt.data'))
    dx_opt.load_state_dict(torch.load('./model/' + dataset + '/dx_opt.data'))
    dy_opt.load_state_dict(torch.load('./model/' + dataset + '/dy_opt.data'))
    
last_epoch = -1
if (load_partial_net):
    last_epoch = CURR_EPOCH-1

gan_lr = torch.optim.lr_scheduler.LambdaLR(gan_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dx_lr = torch.optim.lr_scheduler.LambdaLR(dx_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dy_lr = torch.optim.lr_scheduler.LambdaLR(dy_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)

<h3> Training </h3>

In [9]:
start_time = datetime.datetime.now()
for i in range(CURR_EPOCH,NUM_EPOCHS):
    for j, img in enumerate(imgLoader):
        elapsed_time = datetime.datetime.now()-start_time
        print("Epoch %d/%d, %d/%d, %s"%(i,LR_DECAY_END,j,len(imageSet),str(elapsed_time)))
        src_x = Variable(x.copy_(img['x']))
        src_y = Variable(y.copy_(img['y']))
        
        gan_opt.zero_grad()
        
        #identity Loss
        mapY_Y = G(src_y)
        lossY_Y = identity_loss(mapY_Y, src_y)*5.0
        
        mapX_X = F(src_x)
        lossX_X = identity_loss(mapX_X, src_x)*5.0
        
        #GAN Loss
        mapX_Y = G(src_x) #Generate image Y' from image X
        dX_Y = Dy(mapX_Y) #Check if image Y' seems real
        lossX_Y = gan_loss(dX_Y, dReal) #Calculate Y' realism error
        
        mapY_X = F(src_y) #Generate image X' from image Y
        dY_X = Dx(mapY_X) #Check if image X' seems real
        lossY_X = gan_loss(dY_X, dReal) #Calculate X' realism error
        
        #Cycle Consistency Loss
        mapX_Y_X = F(mapX_Y)
        lossX_Y_X = cycle_loss(mapX_Y_X, src_x)*10.0
        
        mapY_X_Y = G(mapY_X)
        lossY_X_Y = cycle_loss(mapY_X_Y, src_y)*10.0
        
        mapLoss = lossY_Y + lossX_X + lossX_Y + lossY_X + lossX_Y_X + lossY_X_Y
        mapLoss.backward()
        
        gan_opt.step()
        
        #Dx
        dx_opt.zero_grad()
        
        dX_res = Dx(src_x)
        lossX = gan_loss(dX_res, dReal)
        
        mapY_X_old = buffY_X.cycle(mapY_X)
        dY_X = Dx(mapY_X_old.detach())
        lossY_X = gan_loss(dY_X, dFake)
        
        dxLoss = (lossX + lossY_X)*0.5
        dxLoss.backward()
        
        dx_opt.step()
        
        #Dy
        dy_opt.zero_grad()
        
        dY_res = Dy(src_y)
        lossY = gan_loss(dY_res, dReal)
        
        mapX_Y_old = buffX_Y.cycle(mapX_Y)
        dX_Y = Dy(mapX_Y_old.detach())
        lossX_Y = gan_loss(dX_Y, dFake)
        
        dyLoss = (lossY + lossX_Y)*0.5
        dyLoss.backward()
        
        dy_opt.step()     
        
        print("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
        logfile.write("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
    tv.save_image(src_x.data, './img/' + dataset + '/src_x_%d.jpg'%(i))
    tv.save_image(src_y.data, './img/' + dataset + '/src_y_%d.jpg'%(i))
    tv.save_image(mapX_Y.data, './img/' + dataset + '/mapX_Y_%d.jpg'%(i))
    tv.save_image(mapY_X.data, './img/' + dataset + '/mapY_X_%d.jpg'%(i))
    gan_lr.step()
    dx_lr.step()
    dy_lr.step()
    

    if not os.path.isdir("./model/" + dataset "/"):
        os.mkdir("./model/" + dataset + "/")
    model_file = open("./model/" + dataset + "/model_info.txt", "w")
    model_file.write("%d"%(i+1))
    model_file.close()
    torch.save(G.state_dict(), './model/' + dataset + '/G.data')
    torch.save(F.state_dict(), './model/' + dataset + '/F.data')
    torch.save(Dx.state_dict(), './model/' + dataset + '/Dx.data')
    torch.save(Dy.state_dict(), './model/' + dataset + '/Dy.data')
    torch.save(gan_opt.state_dict(), './model/' + dataset + '/gan_opt.data')
    torch.save(dx_opt.state_dict(), './model/' + dataset + '/dx_opt.data')
    torch.save(dy_opt.state_dict(), './model/' + dataset + '/dy_opt.data')

Epoch 91/200, 0/1334, 0:00:00.030259
91,0,1.465819,0.476444,0.001377,0.988245,0.000983

Epoch 91/200, 1/1334, 0:00:02.960690
91,1,1.692085,0.572425,0.000149,1.119345,0.000683

Epoch 91/200, 2/1334, 0:00:03.662527
91,2,1.340293,0.389722,0.000045,0.950491,0.000270

Epoch 91/200, 3/1334, 0:00:04.299388
91,3,2.060521,0.799617,0.003986,1.259945,0.005891

Epoch 91/200, 4/1334, 0:00:04.915442
91,4,2.792332,1.061516,0.170877,1.728118,0.086305

Epoch 91/200, 5/1334, 0:00:05.488089
91,5,1.827965,0.558825,0.017519,1.254293,0.017883

Epoch 91/200, 6/1334, 0:00:06.120311
91,6,1.630942,0.471544,0.020696,1.140366,0.018874

Epoch 91/200, 7/1334, 0:00:06.702483
91,7,1.753674,0.510921,0.016315,1.224669,0.020446

Epoch 91/200, 8/1334, 0:00:07.302316
91,8,1.989595,0.599260,0.007125,1.382063,0.008723

Epoch 91/200, 9/1334, 0:00:07.919280
91,9,1.330480,0.390533,0.000790,0.938155,0.000509

Epoch 91/200, 10/1334, 0:00:08.505685
91,10,1.588547,0.478574,0.002547,1.106028,0.004434

Epoch 91/200, 11/1334, 0:00:09

91,91,2.160592,0.622731,0.000480,1.537820,0.000321

Epoch 91/200, 92/1334, 0:00:57.502501
91,92,2.071608,0.650540,0.000300,1.420432,0.002086

Epoch 91/200, 93/1334, 0:00:58.104026
91,93,1.395361,0.422041,0.000346,0.973309,0.000269

Epoch 91/200, 94/1334, 0:00:58.692817
91,94,1.677088,0.496878,0.000196,1.180168,0.000373

Epoch 91/200, 95/1334, 0:00:59.305613
91,95,2.130737,0.769163,0.000271,1.361515,0.000267

Epoch 91/200, 96/1334, 0:00:59.905432
91,96,1.586701,0.554197,0.000064,1.032504,0.036085

Epoch 91/200, 97/1334, 0:01:00.502539
91,97,1.920628,0.647721,0.000922,1.272391,0.000642

Epoch 91/200, 98/1334, 0:01:01.112322
91,98,1.621486,0.508943,0.003683,1.109519,0.003614

Epoch 91/200, 99/1334, 0:01:01.682669
91,99,1.895110,0.724788,0.006581,1.164829,0.007839

Epoch 91/200, 100/1334, 0:01:02.302415
91,100,2.039287,0.549613,0.002650,1.487595,0.002487

Epoch 91/200, 101/1334, 0:01:02.891925
91,101,2.086721,0.682687,0.000259,1.403532,0.000342

Epoch 91/200, 102/1334, 0:01:03.491675
91,10

91,181,2.775041,0.985160,0.000186,1.789085,0.000467

Epoch 91/200, 182/1334, 0:01:51.591486
91,182,3.067214,1.034295,0.000578,2.031395,0.000897

Epoch 91/200, 183/1334, 0:01:52.182014
91,183,1.761523,0.538217,0.000628,1.222623,0.000342

Epoch 91/200, 184/1334, 0:01:52.800954
91,184,2.204990,0.680393,0.001898,1.523908,0.001273

Epoch 91/200, 185/1334, 0:01:53.406496
91,185,1.805943,0.630892,0.000184,1.174912,0.000215

Epoch 91/200, 186/1334, 0:01:53.987577
91,186,1.686642,0.517569,0.000627,1.168257,0.000329

Epoch 91/200, 187/1334, 0:01:54.595225
91,187,1.797246,0.577269,0.000306,1.219477,0.000595

Epoch 91/200, 188/1334, 0:01:55.172103
91,188,2.084884,0.705027,0.000943,1.378837,0.000856

Epoch 91/200, 189/1334, 0:01:55.807342
91,189,1.916816,0.571209,0.000049,1.345242,0.000153

Epoch 91/200, 190/1334, 0:01:56.414755
91,190,1.784788,0.504856,0.000260,1.279758,0.000134

Epoch 91/200, 191/1334, 0:01:57.003033
91,191,1.530637,0.450616,0.000027,1.079848,0.000289

Epoch 91/200, 192/1334, 0:0

91,271,1.782662,0.762038,0.000601,1.019347,0.000722

Epoch 91/200, 272/1334, 0:02:45.478160
91,272,2.340759,0.836626,0.000426,1.500547,0.001585

Epoch 91/200, 273/1334, 0:02:46.080339
91,273,2.661990,0.983187,0.000474,1.677048,0.000282

Epoch 91/200, 274/1334, 0:02:46.718338
91,274,1.751612,0.486999,0.000398,1.263951,0.000272

Epoch 91/200, 275/1334, 0:02:47.304959
91,275,2.593633,0.698700,0.000372,1.893556,0.000343

Epoch 91/200, 276/1334, 0:02:47.882378
91,276,1.659595,0.477051,0.003176,1.181761,0.001759

Epoch 91/200, 277/1334, 0:02:48.509075
91,277,1.832371,0.553712,0.001627,1.278500,0.001309

Epoch 91/200, 278/1334, 0:02:49.095595
91,278,2.279549,0.694685,0.000111,1.584229,0.000244

Epoch 91/200, 279/1334, 0:02:49.698636
91,279,2.477413,0.811188,0.000178,1.665719,0.000446

Epoch 91/200, 280/1334, 0:02:50.295155
91,280,2.202676,0.794056,0.000812,1.407509,0.001159

Epoch 91/200, 281/1334, 0:02:50.904037
91,281,1.984380,0.629051,0.001513,1.354563,0.001104

Epoch 91/200, 282/1334, 0:0

91,361,1.419278,0.406715,0.000050,1.012416,0.000065

Epoch 91/200, 362/1334, 0:03:39.194940
91,362,1.265597,0.391284,0.000016,0.873828,0.000066

Epoch 91/200, 363/1334, 0:03:39.802222
91,363,1.867905,0.573250,0.000258,1.294136,0.000251

Epoch 91/200, 364/1334, 0:03:40.405150
91,364,1.855371,0.648774,0.000622,1.205647,0.000599

Epoch 91/200, 365/1334, 0:03:40.989777
91,365,2.496427,0.839767,0.000162,1.656062,0.000418

Epoch 91/200, 366/1334, 0:03:41.602137
91,366,2.439728,0.876064,0.000453,1.562713,0.000314

Epoch 91/200, 367/1334, 0:03:42.189679
91,367,2.011004,0.653228,0.000020,1.357438,0.000049

Epoch 91/200, 368/1334, 0:03:42.803086
91,368,1.786135,0.535592,0.000454,1.250114,0.000412

Epoch 91/200, 369/1334, 0:03:43.355589
91,369,1.662701,0.510795,0.000460,1.151621,0.000474

Epoch 91/200, 370/1334, 0:03:43.985877
91,370,2.414215,0.733480,0.000717,1.680412,0.000914

Epoch 91/200, 371/1334, 0:03:44.610508
91,371,1.702234,0.478359,0.000496,1.223804,0.000369

Epoch 91/200, 372/1334, 0:0

...

100,526,1.819758,0.546181,0.000310,1.272324,0.000814

Epoch 100/200, 527/1334, 2:05:17.902080
100,527,2.516991,0.614758,0.000394,1.901690,0.000431

Epoch 100/200, 528/1334, 2:05:18.486676
100,528,2.288484,0.567546,0.000093,1.720692,0.000437

Epoch 100/200, 529/1334, 2:05:19.103489
100,529,2.342125,0.602141,0.000294,1.739691,0.000303

Epoch 100/200, 530/1334, 2:05:19.706160
100,530,1.703320,0.515188,0.000003,1.187575,0.000010

Epoch 100/200, 531/1334, 2:05:20.302598
100,531,1.358871,0.479114,0.000324,0.879580,0.000208

Epoch 100/200, 532/1334, 2:05:20.903072
100,532,2.674073,0.893278,0.000107,1.780276,0.000155

Epoch 100/200, 533/1334, 2:05:21.504817
100,533,2.295640,0.646321,0.000160,1.649258,0.000128

Epoch 100/200, 534/1334, 2:05:22.111806
100,534,2.121135,0.566302,0.000116,1.554375,0.000139

Epoch 100/200, 535/1334, 2:05:22.704569
100,535,2.991106,0.930511,0.000048,2.059814,0.000148

Epoch 100/200, 536/1334, 2:05:23.299584
100,536,1.228933,0.447578,0.000068,0.781044,0.000063

Epoch 

100,614,1.232166,0.425518,0.000164,0.806608,0.000279

Epoch 100/200, 615/1334, 2:06:10.712092
100,615,1.759583,0.517252,0.000078,1.242239,0.000109

Epoch 100/200, 616/1334, 2:06:11.329260
100,616,1.924972,0.700595,0.000002,1.224090,0.000142

Epoch 100/200, 617/1334, 2:06:11.913681
100,617,2.458854,0.829602,0.000222,1.628961,0.000213

Epoch 100/200, 618/1334, 2:06:12.479087
100,618,1.593105,0.487833,0.001114,1.104319,0.000713

Epoch 100/200, 619/1334, 2:06:13.082005
100,619,1.523916,0.510377,0.000008,1.013478,0.000031

Epoch 100/200, 620/1334, 2:06:13.692492
100,620,1.162632,0.428210,0.000276,0.734300,0.000170

Epoch 100/200, 621/1334, 2:06:14.280685
100,621,2.116703,0.665878,0.000425,1.450350,0.001150

Epoch 100/200, 622/1334, 2:06:14.892740
100,622,1.512965,0.478311,0.000072,1.034255,0.000077

Epoch 100/200, 623/1334, 2:06:15.483018
100,623,2.096820,0.607560,0.000383,1.488613,0.001003

Epoch 100/200, 624/1334, 2:06:16.094495
100,624,1.511862,0.389564,0.000045,1.121535,0.000601

Epoch 

100,702,2.187616,0.705616,0.000020,1.481349,0.000420

Epoch 100/200, 703/1334, 2:07:03.508465
100,703,1.267527,0.365498,0.000032,0.901401,0.000110

Epoch 100/200, 704/1334, 2:07:04.107683
100,704,2.068340,0.585124,0.000242,1.482903,0.000142

Epoch 100/200, 705/1334, 2:07:04.707906
100,705,2.004051,0.511034,0.000119,1.492689,0.000089

Epoch 100/200, 706/1334, 2:07:05.293139
100,706,3.235479,0.881822,0.000009,2.353573,0.001031

Epoch 100/200, 707/1334, 2:07:05.904884
100,707,1.398699,0.383710,0.000083,1.014523,0.000120

Epoch 100/200, 708/1334, 2:07:06.498677
100,708,3.366943,1.160853,0.000271,2.205627,0.000187

Epoch 100/200, 709/1334, 2:07:07.108410
100,709,2.203230,0.789360,0.000680,1.413815,0.000430

Epoch 100/200, 710/1334, 2:07:07.707794
100,710,2.967315,0.982709,0.000268,1.984393,0.000206

Epoch 100/200, 711/1334, 2:07:08.300133
100,711,1.530709,0.440179,0.000955,1.090479,0.000675

Epoch 100/200, 712/1334, 2:07:08.900278
100,712,1.471106,0.523287,0.000003,0.947507,0.000267

Epoch 

100,790,2.028496,0.591115,0.000391,1.437370,0.000206

Epoch 100/200, 791/1334, 2:07:56.314502
100,791,1.940204,0.539026,0.000095,1.401080,0.000099

Epoch 100/200, 792/1334, 2:07:56.906894
100,792,1.801779,0.484814,0.000023,1.316952,0.000068

Epoch 100/200, 793/1334, 2:07:57.507421
100,793,1.968187,0.473610,0.000757,1.494572,0.000415

Epoch 100/200, 794/1334, 2:07:58.104374
100,794,2.159852,0.715547,0.000051,1.444224,0.000052

Epoch 100/200, 795/1334, 2:07:58.696729
100,795,2.330279,0.662961,0.000018,1.666893,0.000396

Epoch 100/200, 796/1334, 2:07:59.307684
100,796,1.559609,0.651884,0.000001,0.906712,0.000516

Epoch 100/200, 797/1334, 2:07:59.888613
100,797,1.820142,0.516772,0.000210,1.303162,0.000321

Epoch 100/200, 798/1334, 2:08:00.498375
100,798,2.271567,0.822259,0.000164,1.449306,0.000084

Epoch 100/200, 799/1334, 2:08:01.093542
100,799,1.263343,0.411207,0.000326,0.851820,0.000217

Epoch 100/200, 800/1334, 2:08:01.688702
100,800,1.509793,0.443707,0.000108,1.066060,0.000145

Epoch 

KeyboardInterrupt: 

In [ ]:
close(logfile)